In [9]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nb
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import random
from torch.autograd import Variable
from torch.optim import Adam,SGD
from torchvision import datasets, transforms
import datetime
USE_CUDA = True
random_seed= 42
random.seed( random_seed )
root = os.getcwd()+'/'
print(root)
currentFolder = "3D-Capsule-for-AD/"
data_path=root[:-len(currentFolder)]
print(data_path)


/raid/anagashbayev/3D-Capsule-for-AD/
/raid/anagashbayev/


In [51]:
data_15t_1mm = data_path+'15t/1mm-reg-nii-gz/'
data_15t_2mm = data_path+'15t/2mm-reg-nii-gz/'
data_3t_1mm = data_path+'3t/1mm-reg-nii-gz/'
data_3t_2mm = data_path+'3t/2mm-reg-nii-gz/'
data_type = [data_15t_1mm, data_15t_2mm, data_3t_1mm, data_3t_2mm]
choice = 1 # 0==( 15t 1mm ) 1==(15t 2mm ) 2==( 3t 1mm ) 3==( 3t 2mm ) 
diagnos = {'AD': 0,'MCI': 1,'CN': 2}
val_dia = { 0:'AD', 1:'MCI',2:'CN'}
# diagnos = {'AD': 0,'CN': 1}
# val_dia = { 0:'AD', 1:'CN'}
batch_size = 2#SHOULD BE devisable by 3



n_classes_by_val= len(set(diagnos.values()))
n_classes= len(set(diagnos))
all_data = os.listdir(data_type[choice])
all_subjects = set([i[:10] for i in all_data])
print(f"Total Number of Patients {len(all_subjects)}")
# grouped_data = [[pat for pat in all_data if cls in pat] for cls in diagnos]
grouped_data_imgs = {cls:[pat for pat in all_data if cls in pat] for cls in diagnos}
print("Number of images in each group", {k:len(v) for k,v in grouped_data_imgs.items()})
# grouped_data_imgs

Total Number of Patients 462
Number of images in each group {'AD': 193, 'MCI': 909, 'CN': 442}


In [55]:
grouped_data_sub = {k:list(set([img[:10] for img in v])) for k,v in grouped_data_imgs.items()}
# grouped_data_sub
print("Number of subjects in each group", {k:len(v) for k,v in grouped_data_sub.items()})

Number of subjects in each group {'AD': 89, 'MCI': 212, 'CN': 161}


In [63]:
by_number = False
if by_number:
    test_sub_per_class = 19 
    test_sub = [random.sample(v,test_sub_per_class) for k,v in grouped_data_sub.items()]
else:
    validation_split = .2
    test_sub = [random.sample(v,int(validation_split*len(v))) for k,v in grouped_data_sub.items()]
# test_sub

In [ ]:
train_pat = []
for i in range(n_classes):
    train_pat.append([])
for idx,i in enumerate(grouped_data_pat):
    for idx2,ii in enumerate(i):
        if ii not in test_pat[idx]:
            (train_pat[idx]).append(ii)
        if len(train_pat[idx])==70:
            break
# for i in train_pat:
#     print(len(i))
by_diag = {}
for i in diagnos:
    by_diag[i]={}
for pat in all_data:
    name, image, diagnose =pat.split("__")
    if diagnose in diagnos:
        if name not in by_diag[diagnose]:
            by_diag[diagnose][name]={}
        by_diag[diagnose][name][image]=pat
n_subj_test = [len(i) for i in test_pat]
# n_subj_train  = [len(grouped_data_pat[idx])-n_subj_test[idx] for idx,i in enumerate(by_diag) if i in diagnos]
n_subj_train  = [len(i) for i in train_pat]
n_subj_total = [n_subj_train[i]+n_subj_test[i] for i in range(n_classes)]
# n_subj_test,n_subj_train,n_subj_total
test_list_images_by_dia = [[ii for ii in i if ii[:10] in test_pat[idx] ] for idx,i in enumerate(grouped_data)]
test_tot =0
for i in test_list_images_by_dia:
    test_tot += len(i)
print('Total test images ',test_tot)
train_list_images_by_dia = [[ii for ii in i if ii[:10] in train_pat[idx]] for idx,i in enumerate(grouped_data)]
train_tot = 0
for i in train_list_images_by_dia:
    train_tot += len(i)
print('Total train images ',train_tot)
all_img = []
for i in test_list_images_by_dia:
    all_img = all_img+i
for i in train_list_images_by_dia:
    all_img = all_img+i
class MyData3(Dataset):
    def __init__(self, path,all_imgs):
        self.folder=path
        self.img = all_imgs
        self.len = len(self.img)
    def __len__(self):
        return self.len
    def __getitem__(self, index):
#         temp_size=100
#         vol = torch.zeros(temp_size,temp_size,temp_size,dtype=torch.float32)
        add_noise = False #random.sample([True,False],1)[0]
        di = self.img[index].split('__')[-1]
        name =  os.path.join(self.folder,self.img[index])+'/mri/registered.nii.gz'
        nii = nb.load(name)
        volume = torch.from_numpy(nii.get_fdata()).type(torch.FloatTensor)#[7:82,10:100,10:75]
#         x,y,z = volume.shape
#         if add_noise:
#             noise = 10*torch.randn(volume.shape)
#             volume = volume + noise
#         x,y,z= random.randint(0,temp_size-x),random.randint(0,temp_size-y),random.randint(0,temp_size-z)

        volume = volume/volume.max()
#         vol[x:x+75,y:y+90,z:z+65]=volume
        labels = diagnos[di]
#         return vol.unsqueeze(0), labels
        return volume.unsqueeze(0), labels
# 1: 21-79
# 2: 30-80
# 3: 15-55
dataset=MyData3(data_type[choice],all_img)
test_indices=[i for i in range(test_tot)]
train_indices=[i for i in range(test_tot,test_tot+train_tot)]
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)
train_loader = DataLoader(dataset,sampler=train_sampler, num_workers=10, drop_last=False,batch_size=batch_size)
test_loader = DataLoader(dataset,sampler=test_sampler, num_workers=10, drop_last=False,batch_size=batch_size)